## MicroPython

### 外觀
![esp-d1-mini](image/esp-d1-mini.png)
![esp-lolin](image/esp-lolin.png)

* 這一張圖很重要!!很重要!!很重要!!
* 因為ESP上的D1~D8並不是我們在microPython中的代號，如D4在microPython中為pin2，因此要寫 machine.Pin(2, machine.Pin.OUT) 
<code>
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM
</code>

### MicroPython firmware
- 由這邊下載 Firmware
  http://micropython.org/download#esp8266![image.png](attachment:image.png)
- 安裝燒錄Firmware的工具: esptool.py
- 燒錄流程：
    - Erase  
    esptool.py --port com12 erase_flash
    - 燒錄  
    esptool.py --port com12 --baud 460800 write_flash --flash_size=detect -fm dio 0 py-esp8266-20170108-v1.8.7.bin


### 方便的Serial操作工具: adafruit-ampy    
- 安裝方式  
pip install adafruit-ampy
- put script into board  
ampy --port com12 put test.py
- Run code  
ampy --port com12 run test.py
- Run code forever but no waiting  
ampy --port com12 run test2.py --no-output
- Reset  
ampy --port com12 reset
- List files  
ampy --port com12 ls
- Make dir  
ampy --port com12 mkdir dir3
- Remove dir  
ampy --port com12 rmdir dir3
- Remove file  
ampy --port com12 rm data.txt


## GPIO控制



In [ ]:
# 控制板子上的Led燈
import machine
import time

p = machine.Pin(2, machine.Pin.OUT) #D4

for i in range(6):
    p.value(not p.value())
    time.sleep(1)

## 溫溼度Sensor

- DHT-11 : [datasheet](https://akizukidenshi.com/download/ds/aosong/DHT11.pdf)
![dht-11](image/DHT-11.png)
![dht-11-pin](image/dht-11-pin.png)



In [ ]:
# upy to read temperature, humidity

import dht
import machine

d = dht.DHT11(machine.Pin(0)) #D3
d.measure()
d.temperature() # eg. 23 (℃)
d.humidity()    # eg. 41 (% RH)


## MQTT How to
- MQTT (Message Queue Telemetry Transport)
- 為M2M, IOT設計的協定，基於TCP socket，主要是想讓系統資源較低的設備也可以把資料上傳到Server。
- 目前已是ISO的一員:
    - 2016, MQTT is now an ISO standard (ISO/IEC 20922)
- 本次教學中的MQTT雲端服務為Eclipse所提供：iot.eclipse.org    
- 架構如下：
    - 有興趣可以參考此[文章](https://www.javacodegeeks.com/2016/10/mqtt-protocol-tutorial.html)進一步了解
![mqtt](https://www.javacodegeeks.com/wp-content/uploads/2016/10/mqtt_publisher_subscriber-1.png)


### MQTT Demo請參考 mq-dht11.py
### 使用前請先修改如下：
- TOPIC_BASE = 'iot/home2'
    - 改為你自己的topic名稱
- server = "iot.eclipse.org"
    - 若有自己的MQTT Broker請俢改此行
- wlan.connect('my-ap', '1234567890')    
    - 請改為你的無線網路的 SSID & password
- 一些免費的MQTT服務
    - iot.eclipse.org
    - test.mosquitto.org
    - broker.hivemq.com
    - cloudMqtt (要申請)

In [ ]:
# send T/H sensor data to MQTT cloud platform
# control light with MQTT

from umqtt.simple import MQTTClient
from machine import Pin
import dht
import ubinascii
import machine
import network
import time
import os

# ESP8266 ESP-12 modules have blue, active-low LED on GPIO2
led = Pin(2, Pin.OUT, value=1)
my_new_msg = None
TOPIC_BASE = 'malo-iot'

#Control Function
def led_onoff(onoff):
    """ control led ON or OFF
        parameter:
        onoff
            0-->ON, 1-->OFF (acturely, led ON when level=0)
    """
    global led
    
    if(onoff==1):
        led.value(0)
    elif(onoff==-1):
        led.value(not led.value())
    else:
        led.value(1)

def dht_get():
    ''' get dht11 sensor's value (T, H)
        return:
            (Temperature, Humidity)
    '''
    T=None
    H=None
    try:
        dht11 = dht.DHT11(Pin(0)) #D3

        dht11.measure()
        T = dht11.temperature()
        H = dht11.humidity()
    except Exception as e:
        print('dht_get error:', str(e))
    
    return T, H

    
def sub_cb(topic, msg):
    global my_new_msg
    global TOPIC_BASE
    topic_light = TOPIC_BASE+"/light"
    topic_t = TOPIC_BASE+'/T'
    topic_h = TOPIC_BASE+'/H'

    topic = topic.decode('utf-8')
    msg = msg.decode('utf-8')
    my_new_msg = '['+topic+'] '+ msg
    print(my_new_msg)
    
    if(topic == topic_light):
        if msg == "0":
            led_onoff(0)
        else:
            led_onoff(1)
    if(topic == topic_t):
        pass
    if(topic == topic_h):
        pass
    
def main():
    global my_new_msg
    global TOPIC_BASE
    
    mq_fail_count = 0
    tm_pub_th = time.ticks_ms()

    led_onoff(1)

    #- check ap config file
    AP_SSID = 'upy'
    AP_PWD = 'pypypypy'
    ap_config = None
    ap_config_fn = 'ap.txt'
    if ap_config_fn in os.listdir():
        print('ap config here!')
        f = open(ap_config_fn)
        ap_config = f.read()
        f.close()
    if ap_config:
        print( ('ap_config:', ap_config))
        ap_config = ap_config.split('\n')
        AP_SSID = ap_config[0].strip()
        AP_PWD = ap_config[1].strip()
    print('line to: ', (AP_SSID, AP_PWD))
    
    # Default MQTT server to connect to
    server = "iot.eclipse.org"
    CLIENT_ID = ubinascii.hexlify(machine.unique_id()).decode('utf-8')
    topic_light = TOPIC_BASE+"/light"
    topic_t = TOPIC_BASE+'/T'
    topic_h = TOPIC_BASE+'/H'
    topic_msg = TOPIC_BASE+'/msg'
    

    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    wlan.connect(AP_SSID, AP_PWD)
    print('connecting to AP')
    while(not wlan.isconnected()):
        print(wlan.ifconfig())
        time.sleep(0.1)
        led_onoff(-1)
    print('connected!  --> ', wlan.ifconfig())

    c = MQTTClient(CLIENT_ID, server)
    # Subscribed messages will be delivered to this callback
    c.set_callback(sub_cb)
    c.connect()
    c.subscribe(topic_light)
    print("Connected to %s, subscribed to %s topic" % (server, topic_light))

    # wifi ready, blink led
    for i in range(3):
        led_onoff(1)
        time.sleep(1)
        led_onoff(0)
        time.sleep(1)
    print('I am ready!, ID='+str(CLIENT_ID))
    c.publish(topic_msg, 'I am ready!, ID='+str(CLIENT_ID))

    try:
        while 1:
            if(not wlan.isconnected()):
                # not do any mq operation
                time.sleep(0.1)
                led_onoff(-1)                
                continue
            
            try:
                #c.wait_msg()
                c.check_msg()
                if my_new_msg:
                    c.publish(topic_msg, my_new_msg)
                    my_new_msg = None

                if(time.ticks_ms()-tm_pub_th > 5000):
                    # public some information
                    T, H = dht_get()
                    c.publish(topic_t, str(T))
                    c.publish(topic_h, str(H))
                    
                    tm_pub_th = time.ticks_ms()
                    
            except Exception as e:
                print('wlan:', wlan.isconnected())
                print('ex: ', str(e))
                mq_fail_count+=1
                time.sleep(1)
                
            try:
                if mq_fail_count>5:
                    mq_fail_count=0
                    c = MQTTClient(CLIENT_ID, server)
                    # Subscribed messages will be delivered to this callback
                    c.set_callback(sub_cb)
                    c.connect()
                    c.subscribe(topic_light)
                    print("Connected to %s, subscribed to %s topic" % (server, topic_light))
            except Exception as e:
                print('wlan:', wlan.isconnected())
                print('ex: ', str(e))
                    

            time.sleep(0.001)
                        
    finally:
        c.disconnect()


if __name__ == '__main__':
    main()


# 利用手機和你的IOT設備互動

## 取得溫溼度訊息

- 使用手機為Android手機 (iPhone我沒有，因此沒試過)
- 請安裝「MQTT Dashboard」這個APP
- 裝好後的設定：
    * 裝好後按+新增一個project
    * 填入Server, Port訊息、Client ID任意填
    * ![mqtt-app1](image/mqtt-app1.png)
    * 按+新增一個訂閱(Subscript)的主題(Topic)
    * 填入如圖的內容
    * ![mqtt-app2](image/mqtt-app2.png)
    * 完成畫面如下：
    * ![mqtt-app3](image/mqtt-app3.png)
    * 接著使用相同的方式，增加溼度的訂閱資訊
    * ![mqtt-app4](image/mqtt-app4.png)


## 使用手機控制Led燈
- 按+新增一個public的主題(Topic)
- 選擇Switch
- ![mqtt-app5](image/mqtt-app5.png)
- 填入內容如圖:
- ![mqtt-app6](image/mqtt-app6.png)
- 可以得到畫面如下：
- ![mqtt-app7](image/mqtt-app7.png)
- 此時點switch即可控制upy上的led燈


## 溫溼度資訊只有數字太單調了!?

- 安裝Linear MQTT Dashboard。
- 操作如圖所示
![linear-mqtt1](image/linear-mqtt1.png)



## MQTT 資料遺漏?

- 在Broker的Server上同時有MQTT Client訂閱topic，直接放到DB中!
- 把qos level提升